## installlist


1. pip install requests
1. pip install BeautifulSoup4
1. pip install tqdm

In [ ]:
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup


leaderboard_rl = 'https://lolchess.gg/leaderboards'
res = requests.get(leaderboard_rl)
res.raise_for_status()
leaderboard_soup = BeautifulSoup(res.text,'html.parser')

summoner_list = [] #플레이어 이름 리스트
summoner_number = 100 # 긁어올 플레이어 수
data = [] #데이터 빈 리스트
data_number = 0 # 데이터 개수


# 상위 100 명의 이름 가져오기
for i in tqdm(range(1,summoner_number),desc='gettering top players name'): # tqdm 은 로딩창 구현을 위한것임 , for i in range 와 같은말 ,desc 은 설명
    #선택자 사용
    select = '#wrapper > div.leaderboards.container.mt-4.px-0 > table > tbody > tr:nth-child('+str(i)+') > td.summoner > a' #플레이어 이름 리스트 긁어오기
    summoner = str(leaderboard_soup.select_one(select)) # str 로 변경후 데이터 슬라이싱
    #공백제거 , 띄어쓰기로 분류
    summoner = summoner.replace(' ','').split('\n')
    #리스트에 추가
    summoner_list.append(summoner[1])
    
    
for name in tqdm(summoner_list,desc='gettering top players records'):
    player_rl = 'https://lolchess.gg/profile/kr/'+ name + '/s7/matches/ranked'
    
    
    for k in range(1,10): # 페이지 넘버
        player_rl = 'https://lolchess.gg/profile/kr/'+ name + '/s7/matches/ranked/'+str(k)
        res = requests.get(player_rl)
        res.raise_for_status()
        soup = BeautifulSoup(res.text,'html.parser')
        units = soup.find_all('div',{'class':'units'}) # 페이지 안에 모든 게임 기물에 접근
        rank = soup.find_all('div',{'class':'placement'})

        for i in range(len(units)): # 현재 페이지에 있는 유닛 개수 만큼만 돌아감 (unit(부모 클래스 ) / units(자식 클래스))
            unit = units[i].find_all('div',{'class':'unit'})# 한게임 안에 사용한 게임 기물에 접근
            Ranking = str(rank[i]) # 슬라이싱 하기 위해 데이터 str 형태로 변경
            Ranking = Ranking[24:25] 
            data.append([])
            data[data_number].append(Ranking) # i 값이 아닌 data number로 하는것이 중요

            for j in range(len(unit)): # 유닛 안에있는 자식 클래스 units 개수 만큼 돌아감
                champions = str(unit[j])
                result = champions.find('title=') + 6 # 데이터 슬라이싱 
                champion = champions[result:result+16] # 캐릭터 최대 길이 아우렐리온 솔 고려
                champion = champion.split('"')
                data[data_number].append(champion[1])

            data_number += 1

print(data)

gettering top players records:   2%|█                                                   | 2/99 [00:06<05:13,  3.23s/it]